# Lucas-Kanade Tracking Fisheye Correction Videos - Visual Checks
Last edited: 12/08/2024

By: Christina Wang

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ## next time, load it into your work folder:
# ## dont forget to restart the runtime, so it forgets about the old version !
!cp "/content/drive/My Drive/NASA/cv2_cuda_test/cv2.cpython-310-x86_64-linux-gnu.so" .


%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


#### Check GPU enabled

In [ ]:
import cv2
count = cv2.cuda.getCudaEnabledDeviceCount()
print(count)

1


If not 1, then switch hostruntime environment to GPU environment, or if already on GPU environment, may need to restart session to take effect

## Functions to Conduct Lucas-Kanade Tracking

### Non-Corrected Tracking

In [ ]:
import numpy as np
import cv2 as cv
import pandas as pd

WIDTH_OFFSET = 90 # in pixels
HEIGHT_OFFSET = 20 # in pixels

def optical_flow_uncorrected(video_path, start_time, end_time, center_width, center_height, output_path, min_track_percent=100):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    #print('fps: ', fps)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame
    print('total_frames: ', total_frames)

    #feature_params = dict(maxCorners=1000, qualityLevel=0.3, minDistance=7, blockSize=7)
    feature_params = dict(maxCorners=1000, qualityLevel=0.01, minDistance=7, blockSize=12)
    #lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    #lk_params = dict(winSize=(12, 12), maxLevel=3, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.0001))
    lk_params = dict(winSize=(12, 12), maxLevel=3)
    #color = np.random.randint(0, 255, (2073600, 3))
    #color = np.random.randint(0, 255, (100, 3))
    single_color = np.array([[15, 82, 186]])


    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    ret, old_frame = cap.read()
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    # Start coordinate, here (5, 5)
    # represents the top left corner of rectangle
    #start_point = (660+WIDTH_OFFSET, 240 + HEIGHT_OFFSET) # (w, h) (1920, 1080)
    start_point = (600+WIDTH_OFFSET, 200 + HEIGHT_OFFSET) # (w, h) (1920, 1080)
    start_point = (400, 75)

    # Ending coordinate, here (, 220)
    # represents the bottom right corner of rectangle
    #end_point = (1260 + WIDTH_OFFSET, 840 + HEIGHT_OFFSET)
    end_point = (1250 + WIDTH_OFFSET, 900 + HEIGHT_OFFSET)
    end_point = (1520, 1060)

    center_region = (slice(start_point[1], end_point[1]), slice(start_point[0], end_point[0]))

    # h, w = old_gray.shape
    # center_x, center_y = w // 2, h // 2
    # half_width, half_height = center_width // 2, center_height // 2
    # center_region = (slice(center_y - half_height, center_y + half_height), slice(center_x - half_width, center_x + half_width))
    old_gray = old_gray[center_region]
    h_crop, w_crop = old_gray.shape
    p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    # if p0 is not None:
    #     p0[:, :, 0] += center_x - half_width
    #     p0[:, :, 1] += center_y - half_height

    mask = np.zeros_like(old_frame[center_region])
    color_tracks = {}
    distance_tracker = {}

    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, fps, (w_crop, h_crop))

    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv.CAP_PROP_POS_FRAMES) > end_frame:
            print('No frames grabbed!')
            break


        frame = frame[center_region]

        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                point_id = tuple(map(int, old.ravel())) # store original point location

                distance = np.sqrt((a - c) ** 2 + (b - d) ** 2)

                # mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % 100].tolist(), 2)
                # frame = cv.circle(frame, (int(a), int(b)), 5, color[i % 100].tolist(), -1)
                # color_used = tuple(color[i % 100].tolist())
                # 2073600
                # mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % 2073600].tolist(), 2)
                # frame = cv.circle(frame, (int(a), int(b)), 5, color[i % 2073600].tolist(), -1)
                # color_used = tuple(color[i % 2073600].tolist())
                mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), single_color[0].tolist(), 2)
                frame = cv.circle(frame, (int(a), int(b)), 5, single_color[0].tolist(), -1)
                color_used = tuple(single_color[0].tolist())

                if color_used not in color_tracks:
                  color_tracks[color_used] = [(int(a), int(b)), (int(c), int(d))]
                  distance_tracker[color_used] = distance
                else:
                  color_tracks[color_used].append((int(c), int(d)))
                  distance_tracker[color_used] += distance

            img = cv.add(frame, mask)
            out.write(img)

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    cap.release()
    out.release()

    color_tracks_df = pd.DataFrame(list(color_tracks.items()), columns=['Color', 'Pixel_Positions'])
    color_tracks_df['Track_Length_Frame_Count'] = color_tracks_df['Pixel_Positions'].apply(len)
    color_tracks_df['Track_Percent'] = color_tracks_df['Track_Length_Frame_Count'] / total_frames * 100
    color_tracks_df = color_tracks_df[color_tracks_df['Track_Percent'] >= min_track_percent]

    distance_df = pd.DataFrame(list(distance_tracker.items()), columns=['Color', 'Connect_Dot_Distance'])
    color_tracks_df = pd.merge(color_tracks_df, distance_df, on='Color', how='left')
    color_tracks_df['Travel_Time_Seconds'] = color_tracks_df['Track_Length_Frame_Count'] / fps

    try:
      color_tracks_df['Total_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: np.sqrt((x[0][0] - x[-1][0]) ** 2 + (x[0][1] - x[-1][1]) ** 2))
      color_tracks_df['X_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: (x[-1][0] - x[0][0]))
      color_tracks_df['Y_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: (x[-1][1] - x[0][1]))
      color_tracks_df['Overall_Velocity'] = color_tracks_df['Total_Distance'] / color_tracks_df['Travel_Time_Seconds']
      color_tracks_df['X_Velocity'] = color_tracks_df['X_Distance'] / color_tracks_df['Travel_Time_Seconds']
      color_tracks_df['Y_Velocity'] = color_tracks_df['Y_Distance'] / color_tracks_df['Travel_Time_Seconds']
    except:
      print("none able to be tracked")

    return color_tracks_df

# Example usage:
# df = optical_flow('video.mp4', 10, 20, 12, 12, 'output.avi', min_track_percent=80)
# print(df)


### Corrected Tracking

In [ ]:
import numpy as np
import cv2 as cv
import pandas as pd

def undistort_fisheye_image(distorted_image):
    """
    Apply correction for fisheye distortion.
    Args:
        distorted_image (numpy.ndarray): original image that has the fisheye distortion.
     Returns:
        (numpy.ndarray): undistorted image with fisheye correction.
    """
    # Parameters provided
    f = 1.4  # focal length [mm]
    mu = 2.8e-3  # pixel pitch [mm]
    S = 2  # output (undistorted) image scale factor
    # distortion polynomial order:  [2 4 6 8]
    # polynomial coefficients:
    coeffs = np.array([0.01166363, -0.04819808, 0.07918044, -0.037572])

    H = distorted_image.shape[0]  # image height [pixel]
    W = distorted_image.shape[1]  # image width [pixel]
    cx = (W - 1) / 2  # image center coordinate [pixel]
    cy = (H - 1) / 2  # image center coordinate [pixel]

    K = np.array([[f / mu, 0, cx], [0, f / mu, cy], [0, 0, 1]])

    # compute intrinsic matrix for undistorted image
    cpx = (W * S - 1) / 2
    cpy = (H * S - 1) / 2
    P = np.array([[f / mu, 0, cpx], [0, f / mu, cpy], [0, 0, 1]])

    # rectification matrix (identity)
    R = np.eye(3)

    # produce undistorted image
    map1, map2 = cv.fisheye.initUndistortRectifyMap(K=K, D=coeffs, R=R, P=P, size=[W * S, H * S], m1type=cv.CV_16SC2)
    undistorted_image = cv.remap(distorted_image, map1, map2, interpolation=cv.INTER_LINEAR, borderMode=cv.BORDER_TRANSPARENT)
    return undistorted_image

#def crop_and_correct_image_cv(image, size1=(1500, 1500), offset=(85, 180), size2=(1000, 1000)):
def crop_and_correct_image_cv(image, size1=(1920, 1920), offset=(85, 180), size2=(1500, 1700)):

    h, w = image.shape[:2]
    print("undistorted image h, w:", h, w)

    # Cropping it to 1500X1500 and accounting for offset.
    center_h, center_w = h // 2, w // 2
    offset_h, offset_w = offset
    start_h = min(max(center_h - size1[0] // 2 + offset_h, 0), h - size1[0])
    start_w = min(max(center_w - size1[1] // 2 + offset_w, 0), w - size1[1])
    cropped_image = image[start_h:start_h + size1[0], start_w:start_w + size1[1]]
    h, w = cropped_image.shape[:2]

    # further crop the image to 1000X1000
    center_h, center_w = h // 2, w // 2
    start_h = max(center_h - size2[0] // 2, 0)
    start_w = max(center_w - size2[1] // 2, 0)

    return cropped_image[start_h:start_h + size2[0], start_w:start_w + size2[1]]



def optical_flow_corrected(video_path, start_time, end_time, center_width, center_height, output_path, min_track_percent=100):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    #print('fps: ', fps)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame
    print('total_frames: ', total_frames)

   #feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    #lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    feature_params = dict(maxCorners=1000, qualityLevel=0.01, minDistance=7, blockSize=12)
    lk_params = dict(winSize=(12, 12), maxLevel=3, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.0001))
    # color = np.random.randint(0, 255, (100, 3))
    single_color = np.array([[15, 82, 186]])


    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    ret, old_frame = cap.read()
    #old_color_frame = cv.cvtColor(old_frame, cv.COLOR_BGR2RGB)
    old_color_frame = undistort_fisheye_image(old_frame)
    old_color_frame = crop_and_correct_image_cv(old_color_frame)
    old_gray = cv.cvtColor(old_color_frame, cv.COLOR_BGR2GRAY)

    #old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    h, w = old_gray.shape
    #center_x, center_y = w // 2, h // 2
    #half_width, half_height = center_width // 2, center_height // 2
    #center_region = (slice(center_y - half_height, center_y + half_height), slice(center_x - half_width, center_x + half_width))
    #p0 = cv.goodFeaturesToTrack(old_gray[center_region], mask=None, **feature_params)
    p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    # if p0 is not None:
    #     p0[:, :, 0] += center_x - half_width
    #     p0[:, :, 1] += center_y - half_height

    mask = np.zeros_like(old_color_frame)
    color_tracks = {}
    distance_tracker = {}

    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv.CAP_PROP_POS_FRAMES) > end_frame:
            print('No frames grabbed!')
            break

        #frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        frame = undistort_fisheye_image(frame)
        frame = crop_and_correct_image_cv(frame)
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        #frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                point_id = tuple(map(int, old.ravel())) # store original point location

                distance = np.sqrt((a - c) ** 2 + (b - d) ** 2)

                # mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % 100].tolist(), 2)
                # frame = cv.circle(frame, (int(a), int(b)), 5, color[i % 100].tolist(), -1)
                # color_used = tuple(color[i % 100].tolist())
                mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), single_color[0].tolist(), 2)
                frame = cv.circle(frame, (int(a), int(b)), 5, single_color[0].tolist(), -1)
                color_used = tuple(single_color[0].tolist())

                if color_used not in color_tracks:
                  color_tracks[color_used] = [(int(a), int(b)), (int(c), int(d))]
                  distance_tracker[color_used] = distance
                else:
                  color_tracks[color_used].append((int(c), int(d)))
                  distance_tracker[color_used] += distance

            img = cv.add(frame, mask)
            out.write(img)

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    cap.release()
    out.release()

    color_tracks_df = pd.DataFrame(list(color_tracks.items()), columns=['Color', 'Pixel_Positions'])
    color_tracks_df['Track_Length_Frame_Count'] = color_tracks_df['Pixel_Positions'].apply(len)
    color_tracks_df['Track_Percent'] = color_tracks_df['Track_Length_Frame_Count'] / total_frames * 100
    color_tracks_df = color_tracks_df[color_tracks_df['Track_Percent'] >= min_track_percent]

    distance_df = pd.DataFrame(list(distance_tracker.items()), columns=['Color', 'Connect_Dot_Distance'])
    color_tracks_df = pd.merge(color_tracks_df, distance_df, on='Color', how='left')
    color_tracks_df['Travel_Time_Seconds'] = color_tracks_df['Track_Length_Frame_Count'] / fps

    try:
      color_tracks_df['Total_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: np.sqrt((x[0][0] - x[-1][0]) ** 2 + (x[0][1] - x[-1][1]) ** 2))
      color_tracks_df['X_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: (x[-1][0] - x[0][0]))
      color_tracks_df['Y_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: (x[-1][1] - x[0][1]))
      color_tracks_df['Overall_Velocity'] = color_tracks_df['Total_Distance'] / color_tracks_df['Travel_Time_Seconds']
      color_tracks_df['X_Velocity'] = color_tracks_df['X_Distance'] / color_tracks_df['Travel_Time_Seconds']
      color_tracks_df['Y_Velocity'] = color_tracks_df['Y_Distance'] / color_tracks_df['Travel_Time_Seconds']
    except:
      print("none able to be tracked")

    return color_tracks_df

# Example usage:
# df = optical_flow('video.mp4', 10, 20, 12, 12, 'output.avi', min_track_percent=80)
# print(df)


In [ ]:
def map_coordinates(coord, original_size=(1920, 1080), fisheye_params=None, crop_size1=(1500, 1500), crop_offset=(85, 180), crop_size2=(1000, 1000)):
    """
    Maps a coordinate from the original image to the undistorted image and the crop-corrected image.

    Args:
        coord (tuple): (x, y) coordinate in the original image.
        original_size (tuple): Size of the original image (width, height).
        fisheye_params (dict): Parameters for the fisheye undistortion.
        crop_size1 (tuple): Size of the first crop (width, height).
        crop_offset (tuple): Offset applied to the first crop.
        crop_size2 (tuple): Size of the second crop (width, height).

    Returns:
        dict: A dictionary with mappings to undistorted and crop-corrected coordinates.
    """
    x, y = coord

    # Map from original to undistorted image
    if fisheye_params:
        K = fisheye_params["K"]
        D = fisheye_params["D"]
        R = fisheye_params.get("R", np.eye(3))
        P = fisheye_params["P"]

        # Normalize coordinates to camera space
        coord_homog = np.array([x, y, 1.0], dtype=np.float32).reshape(-1, 1)
        undistorted_coord = cv2.fisheye.undistortPoints(
            coord_homog.T[:, :2].reshape(1, -1, 2),
            K=K, D=D, R=R, P=P
        )[0][0]
        x_undist, y_undist = undistorted_coord
    else:
        x_undist, y_undist = x, y

    # Map from undistorted to crop-corrected image (first crop)
    undist_h, undist_w = original_size[1] * 2, original_size[0] * 2  # Account for fisheye scaling
    center_h, center_w = undist_h // 2, undist_w // 2
    offset_h, offset_w = crop_offset

    # First crop coordinates
    start_h1 = max(center_h - crop_size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - crop_size1[0] // 2 + offset_w, 0)
    x_cropped1 = x_undist - start_w1
    y_cropped1 = y_undist - start_h1

    # Map from first cropped image to final crop-corrected image
    crop1_h, crop1_w = crop_size1
    center_h1, center_w1 = crop1_h // 2, crop1_w // 2

    # Adjust for alignment to center (500, 500)
    start_h2 = center_h1 - crop_size2[1] // 2
    start_w2 = center_w1 - crop_size2[0] // 2
    x_cropped2 = x_cropped1 - start_w2
    y_cropped2 = y_cropped1 - start_h2

    # Return both mapped coordinates
    return {
        "undistorted": (x_undist, y_undist),
        "crop_corrected": (x_cropped2, y_cropped2)
    }

def map_coordinates2(coord, original_size=(1920, 1080), fisheye_params=None, crop_size1=(1500, 1500), crop_offset=(85, 180), crop_size2=(800, 800)):
    """
    Maps a coordinate from the original image to the undistorted image and the crop-corrected image.

    Args:
        coord (tuple): (x, y) coordinate in the original image.
        original_size (tuple): Size of the original image (width, height).
        fisheye_params (dict): Parameters for the fisheye undistortion.
        crop_size1 (tuple): Size of the first crop (width, height).
        crop_offset (tuple): Offset applied to the first crop.
        crop_size2 (tuple): Size of the second crop (width, height).

    Returns:
        dict: A dictionary with mappings to undistorted and crop-corrected coordinates.
    """
    x, y = coord

    # Step 1: Map from original to undistorted image
    if fisheye_params:
        K = fisheye_params["K"]
        D = fisheye_params["D"]
        R = fisheye_params.get("R", np.eye(3))
        P = fisheye_params["P"]

        # Normalize coordinates to camera space
        coord_homog = np.array([x, y, 1.0], dtype=np.float32).reshape(-1, 1)
        undistorted_coord = cv2.fisheye.undistortPoints(
            coord_homog.T[:, :2].reshape(1, -1, 2),
            K=K, D=D, R=R, P=P
        )[0][0]
        x_undist, y_undist = undistorted_coord
    else:
        x_undist, y_undist = x, y  # No undistortion applied

    # Step 2: Map from undistorted to first cropped image
    undist_h, undist_w = original_size[1] * 2, original_size[0] * 2  # Account for fisheye scaling
    center_h, center_w = undist_h // 2, undist_w // 2
    offset_h, offset_w = crop_offset

    # First crop starting points
    start_h1 = max(center_h - crop_size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - crop_size1[0] // 2 + offset_w, 0)
    x_cropped1 = x_undist - start_w1
    y_cropped1 = y_undist - start_h1

    # Step 3: Map from first cropped image to final crop-corrected image
    center_h_crop = center_h - start_h1  # Adjusted center in the first cropped image
    center_w_crop = center_w - start_w1

    # Adjust coordinates for final cropping stage
    start_h2 = max(center_h_crop - crop_size2[1] // 2, 0)
    start_w2 = max(center_w_crop - crop_size2[0] // 2, 0)
    x_cropped2 = x_cropped1 - start_w2
    y_cropped2 = y_cropped1 - start_h2

    # Return the coordinates for both stages
    return {
        "undistorted": (x_undist, y_undist),
        "crop_corrected": (x_cropped2, y_cropped2)
    }

def unmap_coordinates2(crop_corrected_coord, original_size=(1920, 1080), fisheye_params=None,
                       crop_size1=(1500, 1500), crop_offset=(85, 180), crop_size2=(800, 800)):
    """
    Reverses the mapping from crop-corrected coordinates to the undistorted and original image coordinates.

    Args:
        crop_corrected_coord (tuple): (x, y) coordinate in the crop-corrected image.
        original_size (tuple): Size of the original image (width, height).
        fisheye_params (dict): Parameters for the fisheye distortion.
        crop_size1 (tuple): Size of the first crop (width, height).
        crop_offset (tuple): Offset applied to the first crop.
        crop_size2 (tuple): Size of the second crop (width, height).

    Returns:
        dict: A dictionary with mappings to undistorted and original coordinates.
    """
    x_cropped2, y_cropped2 = crop_corrected_coord

    # Step 1: Reconstruct the undistorted coordinates
    undist_h, undist_w = original_size[1] * 2, original_size[0] * 2  # Account for fisheye scaling
    center_h, center_w = undist_h // 2, undist_w // 2
    offset_h, offset_w = crop_offset

    # First crop starting points
    start_h1 = max(center_h - crop_size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - crop_size1[0] // 2 + offset_w, 0)

    # Adjusted center in the first cropped image
    center_h_crop = center_h - start_h1
    center_w_crop = center_w - start_w1

    # Second crop starting points
    start_h2 = max(center_h_crop - crop_size2[1] // 2, 0)
    start_w2 = max(center_w_crop - crop_size2[0] // 2, 0)

    # Reconstruct undistorted coordinates
    x_cropped1 = x_cropped2 + start_w2
    y_cropped1 = y_cropped2 + start_h2
    x_undist = x_cropped1 + start_w1
    y_undist = y_cropped1 + start_h1

    # Step 2: Adjust undistorted coordinates to match the original camera matrix
    if fisheye_params:
        K = fisheye_params["K"]
        D = fisheye_params["D"]
        P = fisheye_params["P"]

        # Compute the offset between the principal points of K and P
        cx_offset = P[0, 2] - K[0, 2]
        cy_offset = P[1, 2] - K[1, 2]

        # Adjust the undistorted coordinates
        x_undist_adjusted = x_undist - cx_offset
        y_undist_adjusted = y_undist - cy_offset

        # Normalize the adjusted undistorted coordinates
        x_norm_undist = (x_undist_adjusted - K[0, 2]) / K[0, 0]
        y_norm_undist = (y_undist_adjusted - K[1, 2]) / K[1, 1]

        # Prepare undistorted normalized coordinates
        undistorted_points = np.array([[[x_norm_undist, y_norm_undist]]], dtype=np.float64)

        # Apply fisheye distortion to get normalized distorted coordinates
        distorted_points = cv2.fisheye.distortPoints(
            undistorted_points,
            K=np.eye(3),
            D=D
        )

        # Map normalized distorted coordinates back to pixel coordinates using K
        x_distorted = distorted_points[0, 0, 0] * K[0, 0] + K[0, 2]
        y_distorted = distorted_points[0, 0, 1] * K[1, 1] + K[1, 2]
        x, y = x_distorted, y_distorted
    else:
        x, y = x_undist, y_undist  # No distortion applied

    # Return the coordinates for both stages
    return {
        "undistorted": (x_undist, y_undist),
        "original": (x, y)
    }


## Functions to get cropped video frames without LK Tracking for comparison

### Un-Corrected Frames

In [ ]:
import numpy as np
import cv2 as cv
import pandas as pd

WIDTH_OFFSET = 90 # in pixels
HEIGHT_OFFSET = 20 # in pixels

def get_plain_video_uncorrected(video_path, start_time, end_time, center_width, center_height, output_path):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    #print('fps: ', fps)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame
    print('total_frames: ', total_frames)

    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    ret, old_frame = cap.read()
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    # Start coordinate, here (5, 5)
    # represents the top left corner of rectangle
    #start_point = (660+WIDTH_OFFSET, 240 + HEIGHT_OFFSET) # (w, h) (1920, 1080)
    start_point = (600+WIDTH_OFFSET, 200 + HEIGHT_OFFSET) # (w, h) (1920, 1080)
    if center_width == 1920:
      start_point = (0, 0)


    # Ending coordinate, here (, 220)
    # represents the bottom right corner of rectangle
    #end_point = (1260 + WIDTH_OFFSET, 840 + HEIGHT_OFFSET)
    end_point = (1250 + WIDTH_OFFSET, 900 + HEIGHT_OFFSET)
    if center_height == 1080:
      end_point = (1920, 1080)
    center_region = (slice(start_point[1], end_point[1]), slice(start_point[0], end_point[0]))

    # h, w = old_gray.shape
    # center_x, center_y = w // 2, h // 2
    # half_width, half_height = center_width // 2, center_height // 2
    # center_region = (slice(center_y - half_height, center_y + half_height), slice(center_x - half_width, center_x + half_width))
    old_gray = old_gray[center_region]
    h_crop, w_crop = old_gray.shape


    mask = np.zeros_like(old_frame[center_region])

    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, fps, (w_crop, h_crop))

    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv.CAP_PROP_POS_FRAMES) > end_frame:
            print('No frames grabbed!')
            break

        frame = frame[center_region]

        img = cv.add(frame, mask)
        out.write(img)

    cap.release()
    out.release()

# Example usage:
# df = optical_flow('video.mp4', 10, 20, 12, 12, 'output.avi', min_track_percent=80)
# print(df)


### Corrected Frames

In [ ]:
def get_plain_video_corrected(video_path, start_time, end_time, center_width, center_height, output_path):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    #print('fps: ', fps)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame
    print('total_frames: ', total_frames)


    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    ret, old_frame = cap.read()
    #old_color_frame = cv.cvtColor(old_frame, cv.COLOR_BGR2RGB)
    old_color_frame = undistort_fisheye_image(old_frame)
    old_color_frame = crop_and_correct_image_cv(old_color_frame)
    old_gray = cv.cvtColor(old_color_frame, cv.COLOR_BGR2GRAY)

    #old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    h, w = old_gray.shape

    mask = np.zeros_like(old_color_frame)


    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv.CAP_PROP_POS_FRAMES) > end_frame:
            print('No frames grabbed!')
            break

        frame = undistort_fisheye_image(frame)
        frame = crop_and_correct_image_cv(frame)
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        img = cv.add(frame, mask)
        out.write(img)

        old_gray = frame_gray.copy()

    cap.release()
    out.release()

## Get Uncropped Video Trimmed to Duration of Interest

In [ ]:
import numpy as np
import cv2 as cv
import pandas as pd

WIDTH_OFFSET = 50 # in pixels
HEIGHT_OFFSET = 40 # in pixels

def get_plain_video_uncropped(video_path, start_time, end_time, output_path):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    #print('fps: ', fps)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame
    print('total_frames: ', total_frames)

    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    ret, old_frame = cap.read()
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    h, w = old_gray.shape



    mask = np.zeros_like(old_frame)

    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv.CAP_PROP_POS_FRAMES) > end_frame:
            print('No frames grabbed!')
            break

        img = cv.add(frame, mask)
        out.write(img)

    cap.release()
    out.release()

# Example usage:
# df = optical_flow('video.mp4', 10, 20, 12, 12, 'output.avi', min_track_percent=80)
# print(df)


## Call Above Functions For Visual Comparisons of 10 Second Snippet of Test Dataset Video

Get the LK tracking on original non-fisheye corrected video.


In [ ]:
#df = optical_flow('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', 10, 20, 630, 720, '/content/drive/MyDrive/NASA/output/cloud_output_non_fisheye_LK_sanity.mp4')
#df = optical_flow_uncorrected('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', 10, 20, 1080, 1920, '/content/drive/MyDrive/NASA/output/full_cloud_output_uncorrected_LK_tracking.mp4')


Get raw frames, no tracking at same cropped area.


In [ ]:
#get_plain_video('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', 10, 20, 630, 720, '/content/drive/MyDrive/NASA/output/cloud_output_non_corrected_plain_LK.mp4')
#get_plain_video_uncorrected('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', 10, 20, 1080, 1920, '/content/drive/MyDrive/NASA/output/full_cloud_output_uncorrected_plain.mp4')



Get LK Tracking on corrected video

In [ ]:
optical_flow_corrected('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', 10, 20, 1080, 1920, '/content/drive/MyDrive/NASA/output/full_cloud_output_corrected_LK_tracking.mp4')


total_frames:  600
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 3840
undistorted image h, w: 2160 

,Color,Pixel_Positions,Track_Length_Frame_Count,Track_Percent,Connect_Dot_Distance,Travel_Time_Seconds,Total_Distance,X_Distance,Y_Distance,Overall_Velocity,X_Velocity,Y_Velocity
0,"(15, 82, 186)","[(1024, 513), (1025, 513), (907, 504), (676, 2...",595219,99203.166667,490985.154351,9920.316667,492.978701,172,462,0.049694,0.017338,0.046571


Get raw frames, corrected, at same cropped area

In [ ]:
get_plain_video_corrected('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', 10, 20, 1080, 1920, '/content/drive/MyDrive/NASA/output/full_cloud_output_corrected_plain.mp4')


total_frames:  600
No frames grabbed!


Get raw frames, uncropped, for perspective

In [ ]:
get_plain_video_uncropped('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', 10, 20, '/content/drive/MyDrive/NASA/output/cloud_output_non_corrected_plain_uncropped_LK.mp4')


total_frames:  600
No frames grabbed!


In [ ]:
df.describe()

,Track_Length_Frame_Count,Track_Percent,Connect_Dot_Distance,Travel_Time_Seconds,Total_Distance,X_Distance,Y_Distance,Overall_Velocity,X_Velocity,Y_Velocity
count,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000
mean,585611.0,97601.833333,129252.170745,9760.183333,413.962559,-394.0,-127.0,0.042413,-0.040368,-0.013012
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,585611.0,97601.833333,129252.170745,9760.183333,413.962559,-394.0,-127.0,0.042413,-0.040368,-0.013012
25%,585611.0,97601.833333,129252.170745,9760.183333,413.962559,-394.0,-127.0,0.042413,-0.040368,-0.013012
50%,585611.0,97601.833333,129252.170745,9760.183333,413.962559,-394.0,-127.0,0.042413,-0.040368,-0.013012
75%,585611.0,97601.833333,129252.170745,9760.183333,413.962559,-394.0,-127.0,0.042413,-0.040368,-0.013012
max,585611.0,97601.833333,129252.170745,9760.183333,413.962559,-394.0,-127.0,0.042413,-0.040368,-0.013012


## Calculate Cloud Heights for Trackable Center Pixels

### Heights for First Frame In Selected 10 second snippet

In [ ]:
import math

diam = 8
frame_pixels = 1080
f = 1.4
v = 202.7
h = 19970
pitch = 0.92

def get_height_helper(u, h, f, v, pitch):
    #print(u, h, f, v, pitch)
    height = (h - f*v/u)*math.cos(math.pi*pitch/180)
    #print('height: ', height)
    return(height)

def calculate_cloud_height(df, v, h, pitch, f=1.4, diam=8, frame_pixels=1080):
  """
  diam = Diameter of the camera frame
  frame_pixels = Pixels of the verticle diameter of the camera frame
  f = focal length of the camera
  v = Velocity of the plane in meters/second
  h = Height of the plane in meters
  """
  df['X_Velocity_Converted'] = df['X_Velocity'] * diam/frame_pixels
  df['Y_Velocity_Converted'] = df['Y_Velocity'] * diam/frame_pixels
  #df['Overall_Velocity_Converted'] = df['Overall_Velocity'] * diam/frame_pixels
  df['Height'] = df['Y_Velocity_Converted'].apply(lambda u: get_height_helper(u, h, f, v, pitch))

  return df

df = calculate_cloud_height(df, v, h, pitch)
df[['Height']]

,Height
0,2.963805e+06


In [ ]:
print(df[['Height']].mean())

Height    2.963805e+06
dtype: float64


### Generic Calculate Cloud Heights for Trackable Center Pixels - Full Test Set Video

#### Set-up Azure Connections for BlobStorage

In [ ]:
!pip install tensorflow-io
!pip install azure-storage-blob azure-identity

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient
import os
import tensorflow as tf
import tensorflow_io as tfio
import pandas as pd

ENV_GOOGLE_COLAB = 1

account_name = ""
account_key = ""
container_name = ""

if ENV_GOOGLE_COLAB:
    from google.colab import userdata

    account_name = userdata.get('storage_account_name')
    account_key = userdata.get('storage_account_key')
    container_name = userdata.get('blob_container_name')
else:
    import configparser

    config = configparser.ConfigParser()
    config.read('../config.ini')
    account_name = config['DEFAULT']['storage_account_name']
    account_key = config['DEFAULT']['storage_account_key']
    container_name = config['DEFAULT']['blob_container_name']

connection_string = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Colab Azure QuickStart Guide: https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/azure.ipynb#scrollTo=Read_and_write_files_to_Azure_Storage_with_TensorFlow
os.environ['TF_AZURE_STORAGE_KEY'] = account_key
# Azure Storage Account Name

# list files and directories in storage container
#tf.io.gfile.listdir(f'az://{account_name}/{container_name}')

#### Get Truth Dataset from Azure Blob Storage

In [ ]:
blobpath = "60fps_v1/20170418/full_df.csv"
client = BlobClient.from_connection_string(connection_string, container_name, blobpath)
blob = client.download_blob()
full_df = pd.read_csv(blob)

In [ ]:
full_df['DateTime_UTC'] = pd.to_datetime(full_df['DateTime_UTC'], infer_datetime_format=True)
truth_df = full_df[full_df['top_height'].notna()]
print("size_before", truth_df.shape)
truth_df = truth_df.loc[truth_df.groupby('timestamp_y')['DateTime_UTC'].idxmin()] # take first frame aggregated at seconds level (not millisecond resolution)
print("size_after", truth_df.shape)
truth_frames = truth_df.frame_num
truth_df.head()

size_before (25864, 52)
size_after (432, 52)


<ipython-input-15-17ea30693ca5>:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  full_df['DateTime_UTC'] = pd.to_datetime(full_df['DateTime_UTC'], infer_datetime_format=True)


,Unnamed: 0,frames_filename,frame_num,Date,Time,DateTime,Short_Name,DateTime_UTC,Lat,Lon,...,lat,lon,alt,angle,N,GH,top_height,bottom_height,layer_discriminator,angle_diff_to_prev
180,180,20170418_175709_frame_180.jpg,180,20170418,175709,2017-04-18 17:57:09,IWG1,2017-04-18 17:57:09.004,34.610995,-86.583988,...,34.6076,-86.5841,19966.0,1.05,2.0,-999.0,13161.0,9953.0,3.0,-0.04
480,480,20170418_175714_frame_480.jpg,480,20170418,175714,2017-04-18 17:57:14,IWG1,2017-04-18 17:57:14.004,34.620100,-86.583985,...,34.6167,-86.5841,19966.0,1.12,2.0,-999.0,13131.0,10013.0,3.0,0.07
780,780,20170418_175719_frame_780.jpg,780,20170418,175719,2017-04-18 17:57:19,IWG1,2017-04-18 17:57:19.004,34.629211,-86.583980,...,34.6258,-86.5841,19966.0,1.12,4.0,210.0,13161.0,9833.0,3.0,0.00
1080,1080,20170418_175724_frame_1080.jpg,1080,20170418,175724,2017-04-18 17:57:24,IWG1,2017-04-18 17:57:24.004,34.638317,-86.583978,...,34.6349,-86.5841,19966.0,1.10,3.0,210.0,13191.0,9923.0,3.0,-0.02
1380,1380,20170418_175729_frame_1380.jpg,1380,20170418,175729,2017-04-18 17:57:29,IWG1,2017-04-18 17:57:29.004,34.647425,-86.583978,...,34.6440,-86.5841,19966.0,1.19,2.0,210.0,13161.0,10133.0,3.0,0.09


In [ ]:
full_df.columns

Index(['Unnamed: 0', 'frames_filename', 'frame_num', 'Date', 'Time',
       'DateTime', 'Short_Name', 'DateTime_UTC', 'Lat', 'Lon', 'GPS_MSL_Alt',
       'WGS_84_Alt', 'Press_Alt', 'Radar_Alt', 'Grnd_Spd', 'True_Airspeed',
       'Indicated_Airspeed', 'Mach_Number', 'Vert_Velocity', 'True_Hdg',
       'Track', 'Drift', 'Pitch', 'Roll', 'Side_slip', 'Angle_of_Attack',
       'Ambient_Temp', 'Dew_Point', 'Total_Temp', 'Static_Press',
       'Dynamic_Press', 'Cabin_Pressure', 'Wind_Speed', 'Wind_Dir',
       'Vert_Wind_Spd', 'Solar_Zenith', 'Sun_Elev_AC', 'Sun_Az_Grd',
       'Sun_Az_AC', 'timestamp_x', 'date', 'timestamp_y', 'lat', 'lon', 'alt',
       'angle', 'N', 'GH', 'top_height', 'bottom_height',
       'layer_discriminator', 'angle_diff_to_prev'],
      dtype='object')

In [ ]:
first_frame_time = full_df[full_df['frame_num'] == 1]['DateTime_UTC'].iloc[0]

truth_df['time_since_first_frame'] = pd.Series(truth_df['DateTime_UTC'] - first_frame_time).dt.total_seconds()

truth_df[['time_since_first_frame']].head()

,time_since_first_frame
180,3.0
480,8.0
780,13.0
1080,18.0
1380,23.0


In [ ]:
truth_df.shape

(432, 53)

In [ ]:
truth_df.tail()

,Unnamed: 0,frames_filename,frame_num,Date,Time,DateTime,Short_Name,DateTime_UTC,Lat,Lon,...,lon,alt,angle,N,GH,top_height,bottom_height,layer_discriminator,angle_diff_to_prev,time_since_first_frame
129840,129840,20170418_183310_frame_129840.jpg,129840,20170418,183310,2017-04-18 18:33:10,IWG1,2017-04-18 18:33:10.004,33.709382,-86.257035,...,-86.2552,19966.0,0.74,2.0,180.0,11962.0,10823.0,3.0,-0.05,2164.0
130140,130140,20170418_183315_frame_130140.jpg,130140,20170418,183315,2017-04-18 18:33:15,IWG1,2017-04-18 18:33:15.004,33.708933,-86.267760,...,-86.2661,19966.0,0.75,2.0,210.0,12022.0,10943.0,3.0,0.01,2169.0
130440,130440,20170418_183320_frame_130440.jpg,130440,20170418,183320,2017-04-18 18:33:20,IWG1,2017-04-18 18:33:20.004,33.708490,-86.278821,...,-86.2770,19966.0,0.74,3.0,240.0,12082.0,10913.0,3.0,-0.01,2174.0
130740,130740,20170418_183325_frame_130740.jpg,130740,20170418,183325,2017-04-18 18:33:25,IWG1,2017-04-18 18:33:25.004,33.708039,-86.289877,...,-86.2879,19966.0,0.75,4.0,240.0,12052.0,10883.0,3.0,0.01,2179.0
131040,131040,20170418_183330_frame_131040.jpg,131040,20170418,183330,2017-04-18 18:33:30,IWG1,2017-04-18 18:33:30.004,33.707594,-86.300582,...,-86.2989,19966.0,0.74,2.0,240.0,12082.0,10853.0,3.0,-0.01,2184.0


#### Get Truth Dataset from Yash's Dataset

In [ ]:
test_df = pd.read_csv("NASA/yash_datasets/train_dataset.csv")
test_df = test_df.dropna()
test_df.head()

,timestamp,image_path,Lat,Lon,GPS_MSL_Alt,WGS_84_Alt,Press_Alt,Grnd_Spd,True_Airspeed,Mach_Number,...,Wind_Speed,Wind_Dir,Solar_Zenith,Sun_Elev_AC,Sun_Az_Grd,Sun_Az_AC,hour_of_day,day_of_year,validation_height,sequence_length
180,2017-04-18 17:57:09,60fps_v1/20170418/170418_175706_183328_frames/...,34.610995,-86.583988,19969.6,19969.8,65045.0,202.8,201.0,0.692,...,3.7,236.4,23.7,67.3,-173.6,-172.5,17.95,108,13161.0,181
181,2017-04-18 17:57:09,60fps_v1/20170418/170418_175706_183328_frames/...,34.610995,-86.583988,19969.6,19969.8,65045.0,202.8,201.0,0.692,...,3.7,236.4,23.7,67.3,-173.6,-172.5,17.95,108,13161.0,1
182,2017-04-18 17:57:09,60fps_v1/20170418/170418_175706_183328_frames/...,34.610995,-86.583988,19969.6,19969.8,65045.0,202.8,201.0,0.692,...,3.7,236.4,23.7,67.3,-173.6,-172.5,17.95,108,13161.0,1
183,2017-04-18 17:57:09,60fps_v1/20170418/170418_175706_183328_frames/...,34.610995,-86.583988,19969.6,19969.8,65045.0,202.8,201.0,0.692,...,3.7,236.4,23.7,67.3,-173.6,-172.5,17.95,108,13161.0,1
184,2017-04-18 17:57:09,60fps_v1/20170418/170418_175706_183328_frames/...,34.610995,-86.583988,19969.6,19969.8,65045.0,202.8,201.0,0.692,...,3.7,236.4,23.7,67.3,-173.6,-172.5,17.95,108,13161.0,1


In [ ]:
test_df.columns

Index(['timestamp', 'image_path', 'Lat', 'Lon', 'GPS_MSL_Alt', 'WGS_84_Alt',
       'Press_Alt', 'Grnd_Spd', 'True_Airspeed', 'Mach_Number',
       'Vert_Velocity', 'True_Hdg', 'Track', 'Drift', 'Pitch', 'Roll',
       'Ambient_Temp', 'Total_Temp', 'Static_Press', 'Dynamic_Press',
       'Cabin_Pressure', 'Wind_Speed', 'Wind_Dir', 'Solar_Zenith',
       'Sun_Elev_AC', 'Sun_Az_Grd', 'Sun_Az_AC', 'hour_of_day', 'day_of_year',
       'validation_height', 'sequence_length'],
      dtype='object')

In [ ]:
# prompt: Can you make the index into a frame_number column and get the first frame_number per timestamp? Do this for the test_df dataframe

# Assuming 'test_df' is already defined as in your provided code.
# Make a copy to avoid modifying the original DataFrame
test_df_copy = test_df.copy()

# Convert the index to a column named 'frame_number'
test_df_copy = test_df_copy.reset_index()
test_df_copy = test_df_copy.rename(columns={'index': 'frame_num'})

# Assuming 'timestamp' is a column in your DataFrame
# Group by 'timestamp' and get the first 'frame_number' for each timestamp
first_frames_per_timestamp = test_df_copy.loc[test_df_copy.groupby('timestamp')['frame_num'].idxmin()]
print(first_frames_per_timestamp.shape)

first_frames_per_timestamp.head()

(432, 32)


,frame_num,timestamp,image_path,Lat,Lon,GPS_MSL_Alt,WGS_84_Alt,Press_Alt,Grnd_Spd,True_Airspeed,...,Wind_Speed,Wind_Dir,Solar_Zenith,Sun_Elev_AC,Sun_Az_Grd,Sun_Az_AC,hour_of_day,day_of_year,validation_height,sequence_length
0,180,2017-04-18 17:57:09,60fps_v1/20170418/170418_175706_183328_frames/...,34.610995,-86.583988,19969.6,19969.8,65045.0,202.8,201.0,...,3.7,236.4,23.7,67.3,-173.6,-172.5,17.95,108,13161.0,181
60,480,2017-04-18 17:57:14,60fps_v1/20170418/170418_175706_183328_frames/...,34.620100,-86.583985,19968.1,19968.3,65032.5,202.8,201.0,...,3.5,236.3,23.7,67.4,-173.6,-172.4,17.95,108,13131.0,241
120,780,2017-04-18 17:57:19,60fps_v1/20170418/170418_175706_183328_frames/...,34.629211,-86.583980,19967.6,19967.6,65032.5,202.8,201.0,...,3.6,237.2,23.7,67.4,-173.5,-172.5,17.95,108,13161.0,241
180,1080,2017-04-18 17:57:24,60fps_v1/20170418/170418_175706_183328_frames/...,34.638317,-86.583978,19967.1,19967.2,65035.0,202.8,200.2,...,4.0,231.4,23.7,67.4,-173.5,-172.5,17.95,108,13191.0,241
240,1380,2017-04-18 17:57:29,60fps_v1/20170418/170418_175706_183328_frames/...,34.647425,-86.583978,19966.6,19966.6,65027.5,202.8,200.8,...,3.9,231.8,23.7,67.4,-173.4,-172.5,17.95,108,13161.0,241


In [ ]:
#truth_df = first_frames_per_timestamp.copy()

#### Get Average Height of Trackable Pixles in Center of Test Dataset Video Frames

In [ ]:
## read previous run

# results_df = pd.read_csv('/content/drive/MyDrive/NASA/output/dataframes/20241027_051119_LK_output.csv')
# print(results_df.shape)
# results_df.head()


In [ ]:
# result.tail()

## Redefine Optical_flow version for center 48 x 48 pixel tracking

In [ ]:
diam = 7.6
frame_pixels = 1082
mu = 7.6/1082
f = 1.4
mu_1 = 2.8e-3
S = 2

def get_undistorted_endpoints(x, y, H, W, mu=mu, S=2):
    cx = (W-1)/2        # image center coordinate [pixel]
    cy = (H-1)/2        # image center coordinate [pixel]

    K = np.array([[f/mu,0,cx],[0,f/mu,cy],[0,0,1]])

    # compute intrinsic matrix for ouput image
    cpx = (W*S-1)/2
    cpy = (H*S-1)/2
    P = np.array([[f/mu,0,cpx],[0,f/mu,cpy],[0,0,1]])

    D = np.array([0.01166363, -0.04819808, 0.07918044, -0.037572])

    R = np.eye(3)

    coord_homog = np.array([x, y, 1.0], dtype=np.float32).reshape(-1, 1)
    undistorted_coord = cv2.fisheye.undistortPoints(
        coord_homog.T[:, :2].reshape(1, -1, 2),
        K=K, D=D, R=R, P=P
    )[0][0]
    x_1, y_1 = undistorted_coord

    return x_1, y_1

In [ ]:
def optical_flow(video_path, start_time, end_time, center_width, center_height, output_path, min_track_percent=100):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    #print('fps: ', fps)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    total_frames = end_frame - start_frame
    print('total_frames: ', total_frames)

    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    color = np.random.randint(0, 255, (100, 3))

    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    ret, old_frame = cap.read()
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    H, W = old_gray.shape[:2]

    h, w = old_gray.shape
    center_x, center_y = w // 2, h // 2
    half_width, half_height = center_width // 2, center_height // 2
    center_region = (slice(center_y - half_height, center_y + half_height), slice(center_x - half_width, center_x + half_width))
    p0 = cv.goodFeaturesToTrack(old_gray[center_region], mask=None, **feature_params)

    if p0 is not None:
        p0[:, :, 0] += center_x - half_width
        p0[:, :, 1] += center_y - half_height

    mask = np.zeros_like(old_frame)
    color_tracks = {}
    distance_tracker = {}

    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv.CAP_PROP_POS_FRAMES) > end_frame:
            print('No frames grabbed!')
            break

        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                point_id = tuple(map(int, old.ravel())) # store original point location

                distance = np.sqrt((a - c) ** 2 + (b - d) ** 2)

                mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % 100].tolist(), 2)
                frame = cv.circle(frame, (int(a), int(b)), 5, color[i % 100].tolist(), -1)
                color_used = tuple(color[i % 100].tolist())

                if color_used not in color_tracks:
                  color_tracks[color_used] = [(int(a), int(b)), (int(c), int(d))]
                  distance_tracker[color_used] = distance
                else:
                  color_tracks[color_used].append((int(c), int(d)))
                  distance_tracker[color_used] += distance

            img = cv.add(frame, mask)
            out.write(img)

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    cap.release()
    out.release()

    color_tracks_df = pd.DataFrame(list(color_tracks.items()), columns=['Color', 'Pixel_Positions'])
    color_tracks_df['Track_Length_Frame_Count'] = color_tracks_df['Pixel_Positions'].apply(len)
    color_tracks_df['Track_Percent'] = color_tracks_df['Track_Length_Frame_Count'] / total_frames * 100
    color_tracks_df = color_tracks_df[color_tracks_df['Track_Percent'] >= min_track_percent]

    distance_df = pd.DataFrame(list(distance_tracker.items()), columns=['Color', 'Connect_Dot_Distance'])
    color_tracks_df = pd.merge(color_tracks_df, distance_df, on='Color', how='left')
    color_tracks_df['Travel_Time_Seconds'] = color_tracks_df['Track_Length_Frame_Count'] / fps

    try:
      color_tracks_df['Total_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: np.sqrt((x[0][0] - x[-1][0]) ** 2 + (x[0][1] - x[-1][1]) ** 2))
      color_tracks_df['Corrected_Start_Pixel'] = color_tracks_df['Pixel_Positions'].apply(lambda x: get_undistorted_endpoints(x[0][0], x[0][1], H, W, mu=mu, S=S))
      color_tracks_df['Corrected_End_Pixel'] = color_tracks_df['Pixel_Positions'].apply(lambda x: get_undistorted_endpoints(x[-1][0], x[-1][1], H, W, mu=mu, S=S))
      color_tracks_df['X_Distance'] = color_tracks_df.apply(lambda row: np.sqrt((row['Corrected_Start_Pixel'][0] - row['Corrected_End_Pixel'][0]) ** 2), axis=1)
      color_tracks_df['Y_Distance'] = color_tracks_df.apply(lambda row: np.sqrt((row['Corrected_Start_Pixel'][1] - row['Corrected_End_Pixel'][1]) ** 2), axis=1)
      # color_tracks_df['X_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: (x[-1][0] - x[0][0]))
      # color_tracks_df['Y_Distance'] = color_tracks_df['Pixel_Positions'].apply(lambda x: (x[-1][1] - x[0][1]))
      # color_tracks_df['Overall_Velocity'] = color_tracks_df['Total_Distance'] / color_tracks_df['Travel_Time_Seconds']
      color_tracks_df['X_Velocity'] = color_tracks_df['X_Distance'] / color_tracks_df['Travel_Time_Seconds']
      color_tracks_df['Y_Velocity'] = color_tracks_df['Y_Distance'] / color_tracks_df['Travel_Time_Seconds']
    except:
      print("none able to be tracked")

    return color_tracks_df

# Example usage:
# df = optical_flow('video.mp4', 10, 20, 12, 12, 'output.avi', min_track_percent=80)
# print(df)

In [ ]:
import time
result_df = None

curr_timestr = time.strftime("%Y%m%d_%H%M%S")

for index, row in truth_df.iterrows():
    frame_number = row['frame_num']
    print("Frame Number: ", frame_number)
    start_time = row['time_since_first_frame']
    end_time = start_time + 10
    center_width = 48
    center_height = 48
    v = row['Grnd_Spd']
    h = row['GPS_MSL_Alt']
    pitch = row['Pitch']
    try:
      df = optical_flow('/content/drive/MyDrive/NASA/input_videos/full_videos/170418_175706_183328.avi', start_time=start_time, end_time=end_time, center_width=center_width, center_height=center_height, output_path=f'/content/drive/MyDrive/NASA/output/frame_tracking_fisheye_corrected_48_48/cloud_tracking_{frame_number}.mp4')
      df = calculate_cloud_height(df, v, h, pitch, f=1.4, diam=8, frame_pixels=1080)
      df['Frame_Number'] = frame_number
      df['True_Lidar_Height'] = row['top_height']
      #df.to_csv(f'/content/drive/MyDrive/NASA/output/dataframes/cloud_tracking_{frame_number}.csv', index=False)
      if result_df is None:
        result_df = df
      else:
        result_df = pd.concat([result_df, df])
      print("Avg Tracked Pixels Height: ", df[['Height']].mean())
      print("True Lidar Height: ", row['top_height'])
    except Exception as e:
      print(e)
      print("No tracking")


Frame Number:  180
total_frames:  600
No frames grabbed!
Avg Tracked Pixels Height:  Height    12571.677467
dtype: float64
True Lidar Height:  13161.0
Frame Number:  480
total_frames:  600
No frames grabbed!
Avg Tracked Pixels Height:  Height    12150.919093
dtype: float64
True Lidar Height:  13131.0
Frame Number:  780
total_frames:  600
No frames grabbed!
Avg Tracked Pixels Height:  Height    12268.455246
dtype: float64
True Lidar Height:  13161.0
Frame Number:  1080
total_frames:  600
No frames grabbed!
Avg Tracked Pixels Height:  Height    12375.560931
dtype: float64
True Lidar Height:  13191.0
Frame Number:  1380
total_frames:  600
No frames grabbed!
Avg Tracked Pixels Height:  Height    12402.799439
dtype: float64
True Lidar Height:  13161.0
Frame Number:  1680
total_frames:  600
No frames grabbed!
Avg Tracked Pixels Height:  Height    12055.95788
dtype: float64
True Lidar Height:  13161.0
Frame Number:  1980
total_frames:  600
No frames grabbed!
Avg Tracked Pixels Height:  Height

In [ ]:
# save calculated LK-based heights for test video
result_df.to_csv(f'/content/drive/MyDrive/NASA/output/dataframes/{curr_timestr}_LK_output_48_48_fisheye_corrected.csv', index=False)


In [ ]:
result_df.shape

(1333, 18)

In [ ]:
result_df.Frame_Number.nunique()

294

In [ ]:
truth_df.frame_num.nunique()

432

In [ ]:
# read latest saved heights csv from this current run
df = pd.read_csv(f'/content/drive/MyDrive/NASA/output/dataframes/{curr_timestr}_LK_output_48_48_fisheye_corrected.csv')
df.head()

,Color,Pixel_Positions,Track_Length_Frame_Count,Track_Percent,Connect_Dot_Distance,Travel_Time_Seconds,Total_Distance,Corrected_Start_Pixel,Corrected_End_Pixel,X_Distance,Y_Distance,X_Velocity,Y_Velocity,X_Velocity_Converted,Y_Velocity_Converted,Height,Frame_Number,True_Lidar_Height
0,"(15, 3, 152)","[(955, 547), (956, 547), (955, 547), (955, 547...",600,100.0,108.239769,10.0,49.040799,"(1914.9972, 1087.0046)","(1912.8224, 1137.544)",2.174805,50.539307,0.217480,5.053931,0.001611,0.037437,12383.638140,180,13161.0
1,"(78, 124, 171)","[(942, 524), (942, 524), (942, 524), (942, 524...",600,100.0,119.434668,10.0,63.126856,"(1901.922, 1063.9309)","(1897.5095, 1128.0836)",4.412476,64.152710,0.441248,6.415271,0.003269,0.047521,13992.744310,180,13161.0
2,"(177, 215, 114)","[(962, 550), (963, 550), (962, 550), (963, 550...",600,100.0,107.023602,10.0,46.173586,"(1922.0023, 1090.0099)","(1917.9596, 1137.524)",4.042725,47.514160,0.404272,4.751416,0.002995,0.035196,11900.850273,180,13161.0
3,"(18, 254, 161)","[(967, 522), (968, 523), (967, 522), (968, 522...",600,100.0,104.666403,10.0,51.156622,"(1927.0221, 1061.9484)","(1931.1188, 1113.346)",4.096680,51.397583,0.409668,5.139758,0.003035,0.038072,12510.262555,180,13161.0
4,"(154, 93, 100)","[(971, 530), (971, 530), (971, 530), (971, 530...",600,100.0,101.307727,10.0,50.358713,"(1931.0208, 1069.9828)","(1937.283, 1120.6549)",6.262207,50.672119,0.626221,5.067212,0.004639,0.037535,12403.512945,180,13161.0


In [ ]:
# Group by the 'Category' column
grouped = df.groupby('Frame_Number', as_index = False)

# Calculate multiple aggregations
result = grouped.agg({
    'Height': ['max', 'min', 'mean'],
    'True_Lidar_Height': 'mean'
})

# results_df['mean_center_height'] = results_df.groupby('Frame_Number', as_index = False)['Height'].mean().Height
# results_df['max_center_height'] = results_df.groupby('Frame_Number', as_index = False)['Height'].max().Height
# results_df['min_center_height'] = results_df.groupby('Frame_Number', as_index = False)['Height'].min().Height
# save max, mean, min heights column if not already existing
#results_df.to_csv('/content/drive/MyDrive/NASA/output/dataframes/20241027_051119_LK_output.csv', index=False)
print(result.shape)

result.head()
result.to_csv(f'/content/drive/MyDrive/NASA/output/dataframes/{curr_timestr}_LK_output_height_min_max_mean_48_48_fisheye_corrected.csv', index=False)


(294, 5)
